In [65]:
from typing import TypedDict, List, Annotated
import operator
import pandas as pd
import requests
import re
import os
import csv
import json
import operator
from typing import Annotated, TypedDict, List
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
import numpy as np

# 初始化基础数据（全局变量）
BASE_DF = pd.read_csv("./X_train.csv")

class AgentState(TypedDict):
    history: List[dict]
    current_idea: str
    current_code: str
    last_score: float
    best_score: float
    iteration: int

In [66]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="lm-studio", # 随便填，本地接口通常不校验
    temperature=0.7
)

In [67]:
def idea_generator(state: AgentState):
    # 1. 整理历史背景
    history_context = ""
    if state.get("history"):
        history_context = "\n".join([f"- 想法: {h['idea']}, 分数: {h.get('score', '未知')}" for h in state['history']])
    else:
        history_context = "尚无历史记录，这是第一次尝试。"

    # 2. 构建 Prompt
    prompt = f"""你是一个资深的数据科学家，正在进行肥料选择预测的特征工程。
    
    【可用基础列名】:
    1. Temparature (注意拼写)
    2. Humidity
    3. Moisture
    4. Nitrogen
    5. Potassium
    6. Phosphorous

    【历史尝试记录】:
    {history_context}

    【任务】:
    请结合加减乘除或非线性组合，提出一个看上去对预测预测肥料类别有用的新因子。
    
    【输出格式要求】:
    请按以下格式输出：
    思考逻辑：<你的分析过程>
    最终想法：<一句话描述因子逻辑，例如：将 Nitrogen 与 Phosphorous 相乘>，非常简单的逻辑即可
    """

    # 3. 调用 LLM 并打印完整输出
    print(f"\n{'='*20} LLM 正在生成新想法 {'='*20}")
    response = llm.invoke(prompt).content.strip()
    print(response) # 打印思考逻辑和想法
    
    # 4. 提取“最终想法”用于后续节点
    # 简单的正则或字符串分割提取
    if "最终想法：" in response:
        idea = response.split("最终想法：")[-1].strip()
    else:
        idea = response

    return {
        "current_idea": idea
    }

In [ ]:
import re
import traceback
import requests
import pandas as pd
import numpy as np
import math

def code_and_eval(state: AgentState):
    idea = state["current_idea"]
    error_message = ""
    current_code = ""
    max_retries = 3  # 代码纠错最大重试次数
    attempt = 0
    score = 0.0

    print(f"\n{'-'*20} 开始代码生成与验证 {'-'*20}")
    print(f"原始想法: {idea}")

    # --- 1. 代码生成与自纠错 (Do-While 逻辑) ---
    while attempt < max_retries:
        # 构造用于代码生成的 Prompt
        if not error_message:
            prompt = f"""你是一个 Python 数据工程师。
            目标：将想法 "{idea}" 转化为一行 pandas 代码。
            可用列名：Temparature, Humidity, Moisture, Nitrogen, Potassium, Phosphorous。
            要求：
            1. 只输出代码，不要任何解释。
            2. 格式必须为：df['新列名'] = ...
            3. 确保使用原始列名拼写（特别是 Temparature）。
            """
        else:
            prompt = f"""你生成的代码出错了。
            代码：{current_code}
            错误：{error_message}
            请分析错误并修复。注意拼写：必须用 Temparature。
            只输出一行修复后的代码。"""

        # 调用 LLM 生成代码
        res = llm.invoke(prompt).content.strip()
        current_code = re.sub(r'```python|```', '', res).strip()
        print(f"尝试 {attempt + 1}: 生成代码 -> {current_code}")

        # 尝试运行代码
        try:
            # 复制一份数据，避免污染 BASE_DF
            df = BASE_DF.copy()
            exec_globals = {"df": df}
            exec(current_code, exec_globals)
            df = exec_globals["df"] # 获取执行后的结果
            
            print("✅ 代码运行成功！")
            error_message = "" # 清空错误信息，跳出循环
            break
        except Exception:
            error_message = traceback.format_exc()
            print(f"❌ 代码运行报错，{error_message}")
            attempt += 1

    # 如果重试多次依然失败
    if error_message:
        print("❌ 达到最大重试次数，代码生成失败。")
        return {"last_score": 0.0, "current_code": "FAILED: " + current_code}

    # --- 2. 调用 API 进行验证 ---
    print(f"🚀 正在请求 API (数据量: {len(df)} 行)...")
    api_url = "http://127.0.0.1:8000/calculate_map3"
    
    try:
        # 注意：这里 75万行转换 JSON 会很慢，建议确认 API 性能
        response = requests.post(api_url, json=df.to_dict(orient='records'), timeout=7200)
        
        if response.status_code == 200:
            raw_res = response.json()["result"]  # 示例: "最高 MAP@3: 0.32"
            try:
                # 按照空格分割，取最后一部分 (即 study_lgb.best_value 的部分)
                score_str = str(raw_res).split(' ')[-1]
                score = float(score_str)
                print(f"📊 验证完成，提取当前最高 MAP@3: {score}")
            except (ValueError, IndexError) as e:
                print(f"⚠️ 解析分数失败，原始返回: {raw_res}，错误: {e}")
                score = 0.0
        else:
            print(f"⚠️ API 请求失败，状态码: {response.status_code}")
            score = 0.0
    except Exception as e:
        print(f"⚠️ API 通信异常: {e}")
        score = 0.0

    # --- 3. 更新分数记录 ---
    best_so_far = state.get("best_score", 0.0)
    new_best = max(best_so_far, score)

    return {
        "current_code": current_code,
        "last_score": score,
        "best_score": new_best
    }

In [69]:
import os

def judge_node(state: AgentState):
    # 准备要记录的内容
    idea = state.get("current_idea", "N/A")
    code = state.get("current_code", "N/A")
    score = state.get("last_score", 0.0)
    best = state.get("best_score", 0.0)
    
    log_line = f"Idea: {idea} | Code: {code} | Score: {score} | Best: {best}\n"
    
    # 写入 .log 文件 (追加模式)
    try:
        with open("experiment.log", "a", encoding="utf-8") as f:
            f.write(log_line)
        print(f"✅ 结果已记录至 experiment.log")
    except Exception as e:
        print(f"❌ 日志写入失败: {e}")

    # 将当前尝试存入 history 供 idea_generator 参考
    new_history = state.get("history", []) + [{"idea": idea, "score": score}]
    
    return {"history": new_history[-5:]}

In [70]:
# --- 5. 构建图 ---
workflow = StateGraph(AgentState)
workflow.add_node("generator", idea_generator)
workflow.add_node("executor", code_and_eval)
workflow.add_node("judge", judge_node)

workflow.set_entry_point("generator")
workflow.add_edge("generator", "executor")
workflow.add_edge("executor", "judge")
workflow.add_conditional_edges("judge", lambda x: "generator" if len(x['history']) < 20 else END)

app = workflow.compile()

In [71]:
import traceback

# 5. 执行启动
if __name__ == "__main__":
    # 初始化状态，必须包含所有 TypedDict 定义的字段
    initial_state = {
        "history": [],
        "current_idea": "",
        "current_code": "",
        "last_score": 0.0,
        "best_score": 0.0,
        "iteration": 0
    }
    
    # recursion_limit 设为 100。
    # 每一轮逻辑包含：Generator -> Executor -> Judge (3个步骤)
    # 100 步约支持 30 轮左右的迭代和纠错
    config = {"recursion_limit": 100}
    
    print("🚀 特征挖掘工作流正式启动...")
    print(f"📈 目标数据规模: {BASE_DF.shape[0]} 行")
    print(f"📝 记录文件: {os.path.abspath('experiment.log')}")
    print("="*50)
    
    try:
        # 核心启动指令
        final_output = app.invoke(initial_state, config=config)
        
        # 运行结束后的汇总展示
        print("\n" + "★"*20 + " 任务执行完毕 " + "★"*20)
        
        if "history" in final_output and final_output['history']:
            # 找到历史记录中分数最高的一项
            valid_history = [h for h in final_output['history'] if isinstance(h.get('score'), (int, float))]
            if valid_history:
                best_feat = max(valid_history, key=lambda x: x['score'])
                print(f"🏆 挖掘出的最佳因子: {best_feat['idea']}")
                print(f"📊 最高 MAP@3 分数: {best_feat['score']}")
                print(f"✅ 详细生成历史已保存至 experiment.log")
            else:
                print("⚠️ 运行完成，但未发现有效的历史分数记录。")
                
    except Exception as e:
        print(f"\n❌ 程序非正常中断!")
        print(f"错误类型: {type(e).__name__}")
        print(f"详细堆栈信息:")
        traceback.print_exc()

🚀 特征挖掘工作流正式启动...
📈 目标数据规模: 750000 行
📝 记录文件: c:\Workspace\面试题目\Agent\experiment.log

==================== LLM 正在生成新想法 ====================
思考逻辑：
在农业中，肥料的选择往往依赖于土壤中的元素平衡。氮（Nitrogen）、磷（Phosphorous）和钾（Potassium）是农作物生长所必需的三大主要元素。然而，过量或不足的任何一种元素都可能影响作物的健康。因此，考虑这些元素之间的相互作用可能是预测肥料类别的一个有效方法。

考虑到这些因素，我们可以尝试通过计算氮、磷和钾的乘积来创建一个新的因子，因为这可以反映土壤中这些元素的理想平衡状态。

最终想法：
将 Nitrogen 与 Phosphorous 相乘

-------------------- 开始代码生成与验证 --------------------
原始想法: 将 Nitrogen 与 Phosphorous 相乘
尝试 1: 生成代码 -> df['Nitrogen_Potassium_Product'] = df['Nitrogen'] * df['Phosphorous']
✅ 代码运行成功！
🚀 正在请求 API (数据量: 750000 行)...
📊 验证完成，MAP@3 分数: 3.0
✅ 结果已记录至 experiment.log

==================== LLM 正在生成新想法 ====================
思考逻辑：为了提高预测肥料类别的准确性，我们可以考虑特征之间的相互作用。在农业中，氮（Nitrogen）、磷（Phosphorous）和钾（Potassium）是主要的营养元素，它们之间存在复杂的相互关系。将这些元素相乘可能会捕捉到它们之间的重要交互效应。

最终想法：将 Nitrogen 与 Phosphorous 相乘

-------------------- 开始代码生成与验证 --------------------
原始想法: 将 Nitrogen 与 Phosphorous 相乘
尝试 1: 生成代码 -> df['Nitrogen_Potassium_Product'] = df['Nitroge

c:\Users\Syswea\.conda\envs\rc\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\Syswea\.conda\envs\rc\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


⚠️ API 通信异常: Out of range float values are not JSON compliant
✅ 结果已记录至 experiment.log

==================== LLM 正在生成新想法 ====================
思考逻辑：

1. 我们观察到之前的一些尝试中，特征的组合和变换并没有得到很好的结果。这可能是由于原始特征之间的关系并不明确，或者特征之间存在冗余信息。

2. 重新审视问题：我们需要一个新因子来预测肥料类别。考虑到历史尝试中的经验，我们可以尝试一些新的组合方式，以捕捉不同特征间的潜在关系。

3. 考虑到 Nitrogen、Phosphorous 和 Potassium 是影响肥料效果的关键成分，我们可以通过它们之间的交互作用来创建一个新的因子。同时，我们也可以考虑其他特征，如 Temperature 和 Humidity，以引入更多的环境因素对肥料效果的影响。

4. 我们可以尝试将 Nitrogen 与 Phosphorous 相乘，并加上 Moisture 的自然对数，以此来反映这些关键成分的交互作用以及土壤湿度对肥料效果的影响。

最终想法：

将 Nitrogen 与 Phosphorous 相乘，并加上 Moisture 的自然对数。

-------------------- 开始代码生成与验证 --------------------
原始想法: 将 Nitrogen 与 Phosphorous 相乘，并加上 Moisture 的自然对数。
尝试 1: 生成代码 -> df['NewColumn'] = df['Nitrogen'] * df['Phosphorous'] + np.log(df['Moisture'])
❌ 代码运行报错
尝试 2: 生成代码 -> import numpy as np
df['NewColumn'] = df['Nitrogen'] * df['Phosphorous'] + np.log(df['Moisture'])
✅ 代码运行成功！
🚀 正在请求 API (数据量: 750000 行)...
📊 验证完成，MAP@3 分数: 3.0
✅ 结果已记录至 experiment.log

==================== LL

KeyboardInterrupt: 